In [1]:
# Simple exercises with the chicago crime dataset from the intro to SQL course
# Set up feedack system

from google.cloud import bigquery
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_crime" dataset
dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Get a reference to the "crime" table
table_ref = dataset_ref.table("crime")

# API request - fetch the table referenced above
table = client.get_table(table_ref)

# Print first 5 rows to see the data
client.list_rows(table,max_results=5).to_dataframe()

In [2]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 gigabytes)
TEN_GB = 10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=TEN_GB)

In [3]:
table.schema

# Questions

1 -What is the year with the highest offences count? (use columns description and year)

2 -What are the 3 most common offences in each year? (use columns description and year)

3 -What are the most common domestic offenses per year? (use columns description and year)

4 -What are the most common offenses that lead to arrest? (use columns description and arrest)

5 -What is the most common hour of day in which offenses are recored? (use field date)it set to choose one)

In [4]:
# 1 -What is the year with the highest offences count? (use columns description and year)
first_query = """
            SELECT count(description) as Num_offences, year as Year
            FROM `bigquery-public-data.chicago_crime.crime`
            GROUP BY Year
            ORDER BY Num_offences DESC
            """
first_query_job = client.query(first_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
first_results = first_query_job.to_dataframe()

# View top few rows of results
print(first_results.head())



In [5]:
import pandas as pd
import pandasql as ps

pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)

## Adjust width of pandas df
pd.set_option('display.width', 1000)

In [6]:
# 2 -What are the 3 most common offences in each year? (use columns description and year)
query_2 = """
            SELECT
            *,
            FROM(
                SELECT
                *, 
                ROW_NUMBER() OVER(PARTITION BY year ORDER BY Num_offences DESC) AS row_number
                FROM( 
                    SELECT year, count(description) as Num_offences, description 
                    FROM `bigquery-public-data.chicago_crime.crime`
                    GROUP BY Year, description
                    )
                )
            WHERE row_number < 4
            ORDER BY Year, Num_offences Desc
            """
query_2_job = client.query(query_2, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
query_2_results = query_2_job.to_dataframe()

# View top few rows of results
print(query_2_results)

In [7]:
# 3 -What are the most common domestic offenses per year? (use columns description and year)
query_2 = """
            SELECT
            *,
            FROM(
                SELECT
                *, 
                ROW_NUMBER() OVER(PARTITION BY year ORDER BY Num_offences DESC) AS row_number
                FROM( 
                    SELECT count(description) as Num_Offences, year, description 
                    FROM `bigquery-public-data.chicago_crime.crime`
                    WHERE description LIKE '%DOMESTIC%' 
                    GROUP BY year, description
                    )
                )
            WHERE row_number = 1
            ORDER BY year, Num_Offences DESC
            """
query_2_job = client.query(query_2, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
query_2_results = query_2_job.to_dataframe()

# View top few rows of results
print(query_2_results)

In [8]:
# 4 -What are the most common offenses that lead to arrest? (use columns description and arrest)
query_2 = """
            SELECT
            *,
            FROM(
                SELECT
                *, 
                ROW_NUMBER() OVER(PARTITION BY year ORDER BY Num_offences DESC) AS row_number
                FROM( 
                    SELECT count(description) as Num_Offences, year, description 
                    FROM `bigquery-public-data.chicago_crime.crime`
                    WHERE arrest = True
                    GROUP BY year, description
                    )
                )
            WHERE row_number = 1
            ORDER BY year, Num_Offences DESC
            """
query_2_job = client.query(query_2, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
query_2_results = query_2_job.to_dataframe()

# View top few rows of results
print(query_2_results)

In [10]:
# 5 -What is the most common hour of day in which offenses are recored? (use field date)
query_2 = """
            SELECT COUNT(1) as Num_Offences, EXTRACT(HOUR FROM date) as Hour_of_day
            FROM `bigquery-public-data.chicago_crime.crime`
            Group by Hour_of_day
            Order by Num_Offences DESC
            LIMIT 5
            """
query_2_job = client.query(query_2, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
query_2_results = query_2_job.to_dataframe()

# View top few rows of results
print(query_2_results)